### getMotif
Returns the extracted set of motifs given the sequences, starting positions and length of motif<br>
@param DNA: set of DNA sequences, Array{String,1}<br>
@param ind: Start positions, Array{Int64,1}<br>
@param k: Length if motif, Int64<br>
@param t: number of sequences, Int64<br>

In [72]:
function getMotif(DNA, ind, k, t)
    sample = Array{String,1}(undef, t)
    for j=1:t
        sample[j] = DNA[j][ind[j]:(ind[j]+k-1)]
    end
    return sample
end

getMotif (generic function with 1 method)

### Score
Returns the normalised consensus score of motif<br>
@param DNA: set of DNA sequences, Array{String,1}<br>
@param ind: Start positions, Array{Int64,1}<br>
@param k: Length if motif, Int64<br>
@param t: number of sequences, Int64<br>

$$ score = \frac{\Sigma max_{b} (f_{b,i})}{l} $$

In [73]:
function Score(DNA, t, k, ind)
    sample = Array{String,1}(undef, t)
    for j=1:t
        sample[j] = DNA[j][ind[j]:(ind[j]+k-1)]
    end
    score = 0
    for i=1:k
        count = Dict('A'=>0,'C'=>0,'G'=>0,'T'=>0)
        for j=1:t
            count[sample[j][i]] = count[sample[j][i]] + 1
        end
        max = 0
        for each in count
            if each[2]>max
                max = each[2]
            end
        end
        score = score+max
    end
    return score/k
end

Score (generic function with 1 method)

### consensus
Prints the consensus motif<br>
@param DNA: set of DNA sequences, Array{String,1}<br>
@param ind: Start positions, Array{Int64,1}<br>
@param k: Length if motif, Int64<br>
@param t: number of sequences, Int64<br>

In [74]:
function consensus(DNA, t, k, ind)
    sample = Array{String,1}(undef, t)
    alphabet = ["A","C","G","T"]
    motif = ""
    for j=1:t
        sample[j] = DNA[j][ind[j]:(ind[j]+k-1)]
    end
    score = 0
    for i=1:k
        count = Dict('A'=>0,'C'=>0,'G'=>0,'T'=>0)
        for j=1:t
            count[sample[j][i]] = count[sample[j][i]] + 1
        end
       print(argmax(count)) 
    end
    println()
end

consensus (generic function with 1 method)

### genIndividual
Returns a candidate motif given number of sequences and upper bound on start position<br>
@param t: number of sequences, Int64<br>
@param ub: Upper bound on motif start position, Int64<br>

In [75]:
function genIndividual(t, ub)
    M = zeros(Int64, 1, t)
    s = rand(t)*(ub)
    for i in 1:t
        s[i] = s[i] + 1
        M[i] = trunc(Int64, s[i])
    end
    return M
end

genIndividual (generic function with 1 method)

### employeeLookAround
Returns a mutated individual given number of sequences and upper bound on start position<br>
Start position in one randomly chosen sequence is changed<br>
@param ind: Start positions, Array{Int64,1}<br>
@param t: number of sequences, Int64<br>
@param ub: Upper bound on motif start position, Int64<br>

In [76]:
function employeeLookAround(ind, t, ub)
    i = trunc(Int64, rand() * t + 1)
    j = trunc(Int64, rand() * ub + 1)
    ind[i] = j
    return ind
end

employeeLookAround (generic function with 1 method)

### crowdingDist
Returns crowding distance for each individual<br>
Crowding distance is calculated as the sum of the sides of the cuboid around an individual that contains no other individual<br>
@param objs: Position of individual described using coloumn vectors, Array{Float64, 2}\[3,t\]<br>

In [77]:
function crowdingDist(objs)
    crowdDist = zeros(Float64, size(objs)[2])
    for i in 1:size(objs)[1]
        v = sortperm(objs[i,:], rev=true)
        crowdDist[v[1]] = crowdDist[v[1]] + 2*objs[i,v[1]]
        crowdDist[v[end]] = crowdDist[v[end]] + 2*objs[i,v[end]]
        for j in 2:length(crowdDist)-1
            crowdDist[v[j]] = crowdDist[v[j]] + (objs[i,v[j-1]] + objs[i,v[j+1]])
        end
    end
    return crowdDist
end

crowdingDist (generic function with 1 method)

### nds
Returns the individuals sorted into dominated collections in descending order<br>
sorting is performed based on values in the 'metric' array<br>
@param metric: Values for sorting, Array{Int64,1}

In [78]:
function nds(metric)
    ndsArchive = Array{Int64,1}[]
    dominated = zeros(Int64, length(metric))
    dominates = Array{Int64}[]
    tempnds = []
    for i in 1:length(metric)
        tempDom = []
        for j in 1:length(metric)
            if metric[i] > metric[j]
                push!(tempDom, j)
            elseif metric[i] < metric[j]
                dominated[i] = dominated[i] + 1
            end
        end
        push!(dominates, tempDom)
        if dominated[i] == 0
            push!(tempnds, i)
        end
    end
    push!(ndsArchive, tempnds)
    i = 1
    while length(ndsArchive[i]) != 0
        tempndsnext = []
        tempndscurr = ndsArchive[i]
        for j in 1:length(tempndscurr)
            p = tempndscurr[j]
            currSet = dominates[p]
            for k in 1:length(currSet)
                q = currSet[k]
                dominated[q] = dominated[q] - 1
                if dominated[q] == 0
                    push!(tempndsnext, q)
                end
            end
        end
        push!(ndsArchive, tempndsnext)
        i = i + 1
    end
    return ndsArchive
end

nds (generic function with 1 method)

### complexity
Returns the entropy of the consensus motif<br>
@param DNA: set of DNA sequences, Array{String,1}<br>
@param ind: Start positions, Array{Int64,1}<br>
@param k: Length if motif, Int64<br>
@param t: number of sequences, Int64<br>
@param prob: Probability of finding each nucleotide in DNA<br>

$$ score = \frac{\Sigma max_{b}(f_{b,i}) log_{2} max_{b}(f_{b,i}) }{p_{b}} $$

In [79]:
function complexity(DNA, t, k, ind, prob)
    sample = Array{String,1}(undef, t)
    for j=1:t
        sample[j] = DNA[j][ind[j]:(ind[j]+k-1)]
    end
    den = 1
    consensusDict = zeros(Int64, 4, 1)
    for i=1:k
        count = Dict('A'=>0,'C'=>0,'G'=>0,'T'=>0)
        for j=1:t
            count[sample[j][i]] = count[sample[j][i]] + 1
        end
        countN = []
        for each in count
            push!(countN, each[2])
        end
        max = 0    
        consensus = i
        for i in 1:4
            if countN[i] > max
                max = countN[i]
                consensus = i
            end
        end
        consensusDict[consensus] = consensusDict[consensus] + 1
    end
    consensusDict = consensusDict .+ 1
    consensusDict = consensusDict ./ (k+4)
    complexityScore = 0
    for i in 1:4
        temp = log(2, consensusDict[i]/prob[i]) * consensusDict[i]
        complexityScore = complexityScore + temp
    end
    return complexityScore
end

complexity (generic function with 1 method)

### getNucProb
Return probability of each nucleotide<br>
@param DNA: Set of DNA sequences, Array{String, 1}<br>

$$ prob_{b} = \frac {instances\ of\ nucleotide\ b} {n*t} $$

In [80]:
function getNucProb(DNA)
    count = Dict('A'=>0,'C'=>0,'G'=>0,'T'=>0)
    n = length(DNA[1])
    t = length(DNA)
    for i in 1:t
        for j in 1:n
            count[DNA[i][j]] = count[DNA[i][j]] + 1
        end
    end
    countN = []
    for each in count
        push!(countN, each[2])
    end
    countSum = sum(countN)
    prob = countN ./ countSum
    return prob
end

getNucProb (generic function with 1 method)

### extract
Given a list of lists, returns a single list containing individual elements<br>
@param archive: List of lists, Array{Any,1}

In [81]:
function extract(archive)
    temp = []
    for each in archive
        append!(temp, each)
    end
    result = Array{Int64, 1}(undef, length(temp))
    for i in 1:length(temp)
        result[i] = temp[i]
    end
    return result
end

extract (generic function with 1 method)

### moabc 
Implementation of the MOABC algorithm<br>
@param DNA: Set of DNA sequences, Array{String, 1}<br>
@param t: number of sequences, Int64<br>
@param N: Size of population, Int64<br>
@param MFE: Number of iterations, Int64<br>

In [88]:
function moabc(DNA, t, N, MFE = 100)
    ndsArchiveExtract = collect(1:N+3)
    ndsArchive = []
    objs=[]
    prob = getNucProb(DNA)
    eval = 0
    n = length(DNA[1])
    Bees = zeros(Int64, N + 3, t)
    fitness = zeros(Float64, N + 3, 1)
    complexityScore = zeros(Float64, N + 3, 1)
    crowdDist = []
    k = zeros(Int64, N+3, 1)
    s = rand(trunc(Int64, N/2))*(64 - 7 + 1)
    for i in 1:trunc(Int64, N/2)
        s[i] = s[i] + 7
        k[i] = trunc(Int64, s[i])
    end
    for i in 1:trunc(Int64, N/2)
        ub = n - k[i] + 1
        temp = genIndividual(t,ub)
        Bees[i,:] = temp
        fitness[i] = Score(DNA, t, k[i], Bees[i,:])
        complexityScore[i] = complexity(DNA, t, k[i], Bees[i,:], prob)
    end
    best = 1
    while eval < MFE
        for i in 1:trunc(Int64, N/2)     # Employee look around phase
            ub = n - k[i] + 1
            temp = employeeLookAround(copy(Bees[i,:]), t, ub)
            tempScore = Score(DNA, t, k[i], temp)
            tempComplexity = complexity(DNA, t, k[i], temp, prob)
            if (tempScore > fitness[i])
                Bees[i,:] = temp
                fitness[i] = tempScore
                complexityScore[i] = tempComplexity
            end
        end
        fitnessSum = sum(fitness[1:trunc(Int64, N/2)])
        probArray = fitness[1:trunc(Int64, N/2)] ./ fitnessSum
        pdfArray = zeros(Float64, trunc(Int64, N/2), 1)
        for i in 1:trunc(Int64, N/2)
            pdfArray[i] = sum(probArray[1:i])
        end
        for i in trunc(Int64, N/2)+1:N      # onlooker phase
            p = rand()
            final = 1                       # employee that has higher fitness has more chance to undergo mutation
            for i in 1:length(pdfArray)
                if (p < pdfArray[i])
                    final = i
                    break
                end
            end
            k[i] = k[final]
            ub = n - k[i] + 1
            temp = employeeLookAround(copy(Bees[final,:]), t, ub)
            tempScore = Score(DNA, t, k[i], temp)
            tempComplexity = complexity(DNA, t, k[i], temp, prob)
            if (tempScore > fitness[final])  # Greedy selection
                Bees[i,:] = temp
                fitness[i] = tempScore
                complexityScore[i] = complexity(DNA, t, k[i], Bees[i,:], prob)
            else
                Bees[i,:] = copy(Bees[final,:])
                fitness[i] = fitness[final]
                complexityScore[i] = complexityScore[final]
            end
        end
        for i in N+1:N+3        # scout phase, no negative feedback
            k[i] = trunc(Int64, rand()*(64 - 7 + 1) + 7)
            ub = n - k[i] + 1
            temp = genIndividual(t,ub)
            Bees[i,:] = temp
            fitness[i] = Score(DNA, t, k[i], Bees[i,:])
            complexityScore[i] = complexity(DNA, t, k[i], Bees[i,:], prob)
        end
        objs = zeros(Float64, 3, N+3)
        objs[1,:] = fitness
        kTemp = k ./ 64
        kTemp = kTemp
        objs[2,:] = kTemp
        objs[3,:] = complexityScore
        crowdDist = crowdingDist(objs)
        ndsArchive = nds(crowdDist)
        ndsArchiveExtract = extract(ndsArchive)
        k[:] = k[ndsArchiveExtract]       # rearrange population
        fitness[:] = fitness[ndsArchiveExtract]
        complexityScore[:] = complexityScore[ndsArchiveExtract]
        #println(Bees)
        #println(crowdDist)
        #println(ndsArchive)
        #println(ndsArchiveExtract)
        for i in 1:size(Bees)[2]
            Bees[:,i] = Bees[:,i][ndsArchiveExtract]
        end
        #println(Bees)
        best = 1
        eval = eval + 1
    end
    println(k[best])
    println(fitness[best])
    #println(complexityScore[best])
    #println(Bees[best,:])
    answer = getMotif(DNA, Bees[best,:], k[best], t)
    consensus(DNA, t, k[best], Bees[best,:])
    return answer
end

moabc (generic function with 2 methods)

### generateInitialPopulation
Returns intial massesfor mogsa<br>
@param DNA: Set of DNA sequences, Array{String, 1}<br>
@param k: array of motif lengths, Array{Int64, 1}<br>
@param t: number of sequences, Int64<br>
@param N: Size of population, Int64<br>

In [89]:
function generateInitialPopulation(DNA,k, t, N)
    startIndices = Array{Int64}(undef,t,N)
    for i=1:t
        for j=1:N
            sIndex = rand(1:length(DNA[i])-k[j]-1)
            startIndices[i, j] = sIndex
        end
    end
    return startIndices
end

generateInitialPopulation (generic function with 1 method)

### mogsa
Implementation of the MOGSA algorithm<br>
@param DNA: Set of DNA sequences, Array{String, 1}<br>
@param t: number of sequences, Int64<br>
@param N: Size of population, Int64<br>
@param MFE: Number of iterations, Int64<br>

In [90]:
using LinearAlgebra

function mogsa(DNA, t, N, timeLimit=200)
    ndsArchive = Array{Int64,1}[]
    currentTime = 0
    kBest = N
    n = length(DNA[1])
    f = zeros(Float64, t, N)
    acc = Array{Float64}(undef, t, N)
    vel = zeros(Float64, t, N)
    k = zeros(Int64, 1, N)
    m = zeros(Float64, 1, N)
    M = zeros(Float64, 1, N)
    fitness = zeros(Float64, 1, N)
    comp = zeros(Float64, 1, N)
    for i=1:N
        k[i] = rand(7:64)
    end
    #print(k)
    x = generateInitialPopulation(DNA,k, t, N)
    G = 100
    alpha = 20
    epsilon = 0.1
           
    while(currentTime < timeLimit)
        #print(currentTime)
        for i=1:N
            fitness[i] = Score(DNA, t, k[i], x[1:t,i])
        end
        prob = getNucProb(DNA)
        for i=1:N
            comp[i] = complexity(DNA, t, k[i], x[1:t,i], prob)
        end
        
        objs = zeros(Float64, 3, N)
        #println(prob)
        objs[1,:] = k ./ 64
        objs[2,:] = fitness
        objs[3,:] = comp
        
        #println(objs[3,:])
        crowdDist = crowdingDist(objs)
        ndsArchive = nds(crowdDist)
        ndsArchiveExtract = extract(ndsArchive)
        k[:] = k[ndsArchiveExtract]
        fitness[:] = fitness[ndsArchiveExtract]
        comp[:] = comp[ndsArchiveExtract]
        for i in 1:size(x)[1]
            x[i,:] = x[i,:][ndsArchiveExtract]
            f[i,:] = f[i,:][ndsArchiveExtract]
            acc[i,:] = acc[i,:][ndsArchiveExtract]
            vel[i,:] = vel[i,:][ndsArchiveExtract]
        end
        #println(p)
        
        #Updating G, Best and Worst
        G = G * exp((-1 * alpha * currentTime)/timeLimit)
        best = maximum(fitness)
        worst = minimum(fitness)
        #println(m)
        #println(best, ' ', worst)
        #Calculate mass of each individual
        for i=1:N
            m[i] = (fitness[i] - worst)/(best - worst)
        end
        #println(m)
        sumOfMass = sum(m)
        for i=1:N
            M[i] = m[i]/sumOfMass
        end
        #println(M)
        #Calculate force and acceleration on each individual 63
        for d=1:t
            fTemp = zeros(Float64,N,N)
            for i=1:N
                for j=1:kBest
                    r = norm(x[d,i]-x[d,j])
                    fTemp[i,j] = (G * ((M[i] * M[j])/(r + epsilon)) * (x[d,i]-x[d,j]))
                end
                #println(fTemp)
                for j=1:kBest
                    if j!=i
                       f[d,i] = f[d,i] + rand(0:1) * fTemp[i,j] 
                    end
                end
                acc[d,i] = f[d,i]/(M[i] + 0.00001)
            end
        end
        #println(acc)
        #Calculate velocity and position of each individual
        #print(vel)
        for d=1:t
            for i=1:N
                vel[d,i] = rand(0:1) * vel[d,i] + acc[d,i]
                temp = convert(Int, floor(x[d,i] + vel[d,i]))
                if (temp < 1) || (temp > n-k[i]-1)
                    x[d,i] = x[d,i]
                else
                    x[d, i] = temp
                end
            end
        end
        #print(x)
        #print(NDSArchive)
        kBest = kBest - length(ndsArchive[end-1])
        #print(kbest)
        currentTime = currentTime + 1
    end
    #Print Motifs
    #println(k[ndsArchive[1]])
    #println(transpose(x[:,ndsArchive[1]]))
    println(k[ndsArchive[1]][1])
    println(fitness[ndsArchive[1]][1])
    consensus(DNA, t, k[ndsArchive[1]][1], x[:,ndsArchive[1]])
    return getMotif(DNA, transpose(x[:,ndsArchive[1]]), k[ndsArchive[1]][1], t)
end

mogsa (generic function with 2 methods)

### mmoabc 
Implementation of the MOABC algorithm with constraints on scouts<br>
@param DNA: Set of DNA sequences, Array{String, 1}<br>
@param t: number of sequences, Int64<br>
@param N: Size of population, Int64<br>
@param MFE: Number of iterations, Int64<br>

In [91]:
function mmoabc(DNA, t, N, MFE = 100)
    ndsArchiveExtract = collect(1:N+3)
    ndsArchive = []
    prob = getNucProb(DNA)
    eval = 0
    n = length(DNA[1])
    Bees = zeros(Int64, N + 3, t)
    fitness = zeros(Float64, N + 3, 1)
    complexityScore = zeros(Float64, N + 3, 1)
    crowdDist = []
    k = zeros(Int64, N+3, 1)
    s = rand(trunc(Int64, N/2))*(64 - 7 + 1)
    for i in 1:trunc(Int64, N/2)
        s[i] = s[i] + 7
        k[i] = trunc(Int64, s[i])
    end
    for i in 1:trunc(Int64, N/2)        # Generate initial population of employees
        ub = n - k[i] + 1
        temp = genIndividual(t,ub)
        Bees[i,:] = temp
        fitness[i] = Score(DNA, t, k[i], Bees[i,:])
        complexityScore[i] = complexity(DNA, t, k[i], Bees[i,:], prob)
    end
    best = 1
    while eval < MFE
        for i in 1:trunc(Int64, N/2)     # Employee phase
            ub = n - k[i] + 1
            temp = employeeLookAround(copy(Bees[i,:]), t, ub)
            tempScore = Score(DNA, t, k[i], temp)
            tempComplexity = complexity(DNA, t, k[i], temp, prob)
            if (tempScore > fitness[i])
                Bees[i,:] = temp
                fitness[i] = tempScore
                complexityScore[i] = tempComplexity
            end
        end
        fitnessSum = sum(fitness[1:trunc(Int64, N/2)])
        probArray = fitness[1:trunc(Int64, N/2)] ./ fitnessSum
        pdfArray = zeros(Float64, trunc(Int64, N/2), 1)
        for i in 1:trunc(Int64, N/2)
            pdfArray[i] = sum(probArray[1:i])
        end
        for i in trunc(Int64, N/2)+1:N      # onlooker phase
            p = rand()                      # Higher the fitness of the individual,
                                            # more likely is it to be selected for mutation
            final = 1
            for i in 1:length(pdfArray)
                if (p < pdfArray[i])
                    final = i
                    break
                end
            end
            k[i] = k[final]
            ub = n - k[i] + 1
            temp = employeeLookAround(copy(Bees[final,:]), t, ub)
            tempScore = Score(DNA, t, k[i], temp)
            tempComplexity = complexity(DNA, t, k[i], temp, prob)
            if (tempScore > fitness[final])           # greedy selection
                Bees[i,:] = temp
                fitness[i] = tempScore
                complexityScore[i] = complexity(DNA, t, k[i], Bees[i,:], prob)
            else
                Bees[i,:] = copy(Bees[final,:])
                fitness[i] = fitness[final]
                complexityScore[i] = complexityScore[final]
            end
        end
        maxFitness = argmax(fitness).I[1]
        thresh = (maximum(fitness) + minimum(fitness)) / 2 # threshold to generate negative feedback
        for i in N+1:N+3        # scout phase
            kTemp= trunc(Int64, rand()*(64 - 7 + 1) + 7)
            ub = n - kTemp + 1
            temp = genIndividual(t,ub)
            tempScore = Score(DNA, t, kTemp, temp)
            if tempScore > thresh
                Bees[i,:] = temp
                k[i] = kTemp
            else
                Bees[i,:] = Bees[maxFitness,:]
                k[i] = k[maxFitness]
            end
            fitness[i] = Score(DNA, t, k[i], Bees[i,:])
            complexityScore[i] = complexity(DNA, t, k[i], Bees[i,:], prob)
        end
        
        objs = zeros(Float64, 3, N+3)
        objs[1,:] = fitness
        kTemp = k ./ 64
        kTemp = kTemp
        objs[2,:] = kTemp
        objs[3,:] = complexityScore 
        crowdDist = crowdingDist(objs)
        ndsArchive = nds(crowdDist)
        ndsArchiveExtract = extract(ndsArchive)
        k[:] = k[ndsArchiveExtract]         # rearranging the population
        fitness[:] = fitness[ndsArchiveExtract]
        complexityScore[:] = complexityScore[ndsArchiveExtract]
        #println(Bees)
        #println(crowdDist)
        #println(ndsArchive)
        #println(ndsArchiveExtract)
        for i in 1:size(Bees)[2]
            Bees[:,i] = Bees[:,i][ndsArchiveExtract]
        end
        #println(Bees)
        best = 1
        eval = eval + 1
    end
    println(k[best])
    println(fitness[best])
    #println(complexityScore[best])
    #println(Bees[best,:])
    consensus(DNA, t, k[best], Bees[best,:])
    answer = getMotif(DNA, Bees[best,:], k[best], t)
    return answer
end

mmoabc (generic function with 2 methods)

In [92]:
open("test1.txt") do file
    x = readline(file)
    line1 = split(x," ")
    t = parse(Int64, line1[1])
    DNA = Array{String,1}(undef,t)
    for i in 1:t
        DNA[i] = readline(file)
    end
    println(length(DNA[1]))
    #@time Motif1 = abc(DNA, k, t, N, 50, 500)
    #for each in Motif1
    #    println(each)
    #end
    println(t*3/4)
    @time Motif1 = moabc(DNA, t, 200, 200)
    for each in Motif1
        println(each)
    end
    @time Motif2 = mogsa(DNA, t, 200, 200)
    for each in Motif2
        println(each)
    end
    @time Motif3 = mmoabc(DNA, t, 200, 200)
    for each in Motif3
        println(each)
    end
end

200
3.75
37
4.648648648648648
CGGGCCGGGCGCAGGGCCGGCGGGCCGCGCCCCCGCG
  1.603345 seconds (15.95 M allocations: 1.802 GiB, 15.72% gc time)
CGGGCCGGGCGCAGGGCCGGCGGGCCGCGCCGGGGCG
CGGGCCGGGCGCAGGGCCGGCGGGCCGCGCTCCCGCC
CGGGCCGGGCGCAGGGCCGGCGGGCCGCGCGCCCGTA
CGGGCCGGGCGCAGGGCCGGCGGGCCGCGCCCCCAAG
CGGGCCGGGCGCAGGGCCGGCGGGCCGCGCCTCGGCC
64
3.0
CGCGGGCCGCGGGGCGCGGGGGCGGGCAGGGCCGGCCGGCCCCGGGGCGCGGCGGGCGCGACGG
  2.131817 seconds (15.83 M allocations: 2.173 GiB, 13.10% gc time)
GAGTGCCCGGAGGCCGCGGGGTCGGGCGCACCCGGCCTTCCACGGGTCGCCAGGGCGGCGAGGG
CCCGCGCTGCCGGCCCGTGAGTCCAGCAGCCTCGGCAGGCGCCGGCAGGCGGCCGCGGCGGCCC
CCCCCGGTGAGGAGGGCCCGTGCCGCCGGGGGGTCCCGCACGGGGTTTCCATCGAGCCACATGG
AGCCTTCCACGGGTCGCCAGGGCGGCGAGGGCGCGTCGGGCCGGGCGCAGGGCCGGCGGGCCGC
CGGGGGCCCTGGCGGGGGGGGGATGACTCTTCCACGGGTCGCCAGGGCGGCGAGGGCGCGTCGG
62
4.370967741935484
CGGCGAGGGCGCGTCGGGCCGGGCGCAGGGCCGGCGGGCCGCGCCCCCGCGTGGGAGGCCCC
  2.414673 seconds (26.10 M allocations: 2.961 GiB, 16.46% gc time)
CGGCGAGGGCGCGTCGGGCCGGGCGCAGGGCCGGCGGGCCGCGCCGGGGCGGGACTT

In [87]:
count = Dict('A'=>5,'C'=>0,'G'=>0,'T'=>0)

ErrorException: cannot assign variable Base.count from module Main

In [23]:
argmax(count)

'A': ASCII/Unicode U+0041 (category Lu: Letter, uppercase)